In [18]:
import pandas as pd

from ast import literal_eval
from IPython.display import display
from ipywidgets import interact, widgets
from rdkit.Chem import Draw, Mol, PandasTools

from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.molSize = (600, 600)

In [19]:
substrates = PandasTools.LoadSDF('/home/rjacob/SOMAN/output/metatrans/substrates.sdf')
metabolites = PandasTools.LoadSDF('/home/rjacob/SOMAN/output/metatrans/metabolites.sdf')
data = pd.merge(substrates, metabolites, on=['ID', 'substrate_id', 'metabolite_id', 'substrate_name', 'metabolite_name', 'soms'], how='inner', suffixes=('_substrate', '_metabolite'))
data['soms'] = data['soms'].apply(literal_eval)

[13:28:01] The 2 defining bonds for an atropisomer are co-planar - atoms are: 30 32
[13:28:01] The 2 defining bonds for an atropisomer are co-planar - atoms are: 19 21
[13:28:01] The 2 defining bonds for an atropisomer are co-planar - atoms are: 13 15
[13:28:01] The 2 defining bonds for an atropisomer are co-planar - atoms are: 30 32
[13:28:01] The 2 defining bonds for an atropisomer are co-planar - atoms are: 32 23
[13:28:01] The 2 defining bonds for an atropisomer are co-planar - atoms are: 23 29
[13:28:01] The 2 defining bonds for an atropisomer are co-planar - atoms are: 13 15
[13:28:01] The 2 defining bonds for an atropisomer are co-planar - atoms are: 11 13
[13:28:02] The 2 defining bonds for an atropisomer are co-planar - atoms are: 4 5


In [20]:
# data_filtered = data[data['annotation_rule'] != 'maximum number of heavy atoms filter']
# data_filtered_shuffled = data_filtered.sample(frac=1).reset_index(drop=True)

@interact(i=widgets.IntSlider(min=0, max=len(data)-1, step=1))
def f(i):

    row= data.iloc[i]

    substrate = Mol(row.ROMol_substrate)
    substrate.Compute2DCoords()
    for atom in substrate.GetAtoms():
        atom.SetIntProp("atomNote", atom.GetIdx())

    metabolite = Mol(row.ROMol_metabolite)
    metabolite.Compute2DCoords()
    for atom in metabolite.GetAtoms():
        atom.SetIntProp("atomNote", atom.GetIdx())

    print(f"Substrate: {row.substrate_id}")
    print(f"Metabolite: {row.metabolite_id}")
    print(f"Predicted SoMs: {row.soms}")
    print(f"Annotation rule: {row.annotation_rule}")

    display(Draw.MolsToGridImage([substrate, metabolite], 
                                  molsPerRow=2, 
                                  subImgSize=(500, 500), 
                                  legends=["substrate", "metabolite"], 
                                  highlightAtomLists=[row.soms, []], 
                                  highlightBondLists=[]))

interactive(children=(IntSlider(value=0, description='i', max=9081), Output()), _dom_classes=('widget-interact…